In [136]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics

Data Collection and Processing

In [137]:
# load data
dataset = pd.read_csv('sales_dataset.csv')
dataset

c:\Users\Asus\anaconda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,order_id,order_date,status,item_id,sku,qty_ordered,price,value,discount_amount,total,...,SSN,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Discount_Percent
0,100354678,2020-10-01,received,574772.0,oasis_Oasis-064-36,22.0,89.9,1798.0,0.0,1798.0,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
1,100354678,2020-10-01,received,574774.0,Fantastic_FT-48,12.0,19.0,190.0,0.0,190.0,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
2,100354680,2020-10-01,complete,574777.0,mdeal_DMC-610-8,10.0,149.9,1199.2,0.0,1199.2,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
3,100354680,2020-10-01,complete,574779.0,oasis_Oasis-061-36,10.0,79.9,639.2,0.0,639.2,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
4,100367357,2020-11-13,received,595185.0,MEFNAR59C38B6CA08CD,3.0,99.9,99.9,0.0,99.9,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286387,100562365,2021-09-30,paid,905179.0,APPCHA5AF14939B8F8A,3.0,4419.9,4419.9,0.0,4419.9,...,613-87-0361,212-772-7404,Rushville,Yates,Rushville,NY,14544,Northeast,bklatham,0.0
286388,100562376,2021-09-30,cod,905191.0,MEFCOT5A8D1E973B886,3.0,39.9,39.9,0.0,39.9,...,101-02-1040,229-817-9451,Lawrenceville,Gwinnett,Lawrenceville,GA,30044,South,bmbrunetti,0.0
286389,100562383,2021-09-30,cod,905200.0,WOFVAL59D5EA84167F9-M,3.0,40.0,40.0,0.0,40.0,...,399-31-7238,252-414-8396,Durham,Durham,Durham,NC,27701,South,fngiusti,0.0
286390,100562384,2021-09-30,cod,905202.0,WOFNIG5B4D7EB0E9FDD-L,3.0,49.9,49.9,0.0,49.9,...,399-31-7238,252-414-8396,Durham,Durham,Durham,NC,27701,South,fngiusti,0.0


In [138]:
# remove duplicate entries
print(dataset.shape)
dataset.drop_duplicates(inplace = True)
dataset.shape

(286392, 36)


(286392, 36)

Create new dataset with relevant features

In [139]:
dataset = dataset[['month', 'category', 'Gender', 'age', 'Region', 'State', 'price', 'value', 'Discount_Percent', 'discount_amount',  'total', 'qty_ordered']]
dataset

,month,category,Gender,age,Region,State,price,value,Discount_Percent,discount_amount,total,qty_ordered
0,Oct-2020,Men's Fashion,F,43.0,South,OK,89.9,1798.0,0.0,0.0,1798.0,22.0
1,Oct-2020,Men's Fashion,F,43.0,South,OK,19.0,190.0,0.0,0.0,190.0,12.0
2,Oct-2020,Men's Fashion,F,43.0,South,OK,149.9,1199.2,0.0,0.0,1199.2,10.0
3,Oct-2020,Men's Fashion,F,43.0,South,OK,79.9,639.2,0.0,0.0,639.2,10.0
4,Nov-2020,Men's Fashion,F,43.0,South,OK,99.9,99.9,0.0,0.0,99.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
286387,Sep-2021,Appliances,M,51.0,Northeast,NY,4419.9,4419.9,0.0,0.0,4419.9,3.0
286388,Sep-2021,Men's Fashion,M,52.0,South,GA,39.9,39.9,0.0,0.0,39.9,3.0
286389,Sep-2021,Women's Fashion,F,38.0,South,NC,40.0,40.0,0.0,0.0,40.0,3.0
286390,Sep-2021,Women's Fashion,F,38.0,South,NC,49.9,49.9,0.0,0.0,49.9,3.0


In [140]:
# seperate month and year
dataset[['month', 'year']] = dataset.month.str.split("-", expand=True)
dataset.drop('year', axis=1, inplace=True)

c:\Users\Asus\anaconda3\envs\pytorch\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
c:\Users\Asus\anaconda3\envs\pytorch\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [141]:
dataset

,month,category,Gender,age,Region,State,price,value,Discount_Percent,discount_amount,total,qty_ordered
0,Oct,Men's Fashion,F,43.0,South,OK,89.9,1798.0,0.0,0.0,1798.0,22.0
1,Oct,Men's Fashion,F,43.0,South,OK,19.0,190.0,0.0,0.0,190.0,12.0
2,Oct,Men's Fashion,F,43.0,South,OK,149.9,1199.2,0.0,0.0,1199.2,10.0
3,Oct,Men's Fashion,F,43.0,South,OK,79.9,639.2,0.0,0.0,639.2,10.0
4,Nov,Men's Fashion,F,43.0,South,OK,99.9,99.9,0.0,0.0,99.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
286387,Sep,Appliances,M,51.0,Northeast,NY,4419.9,4419.9,0.0,0.0,4419.9,3.0
286388,Sep,Men's Fashion,M,52.0,South,GA,39.9,39.9,0.0,0.0,39.9,3.0
286389,Sep,Women's Fashion,F,38.0,South,NC,40.0,40.0,0.0,0.0,40.0,3.0
286390,Sep,Women's Fashion,F,38.0,South,NC,49.9,49.9,0.0,0.0,49.9,3.0


In [142]:
# getting some information about the dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286392 entries, 0 to 286391
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   month             286392 non-null  object 
 1   category          286392 non-null  object 
 2   Gender            286392 non-null  object 
 3   age               286392 non-null  float64
 4   Region            286392 non-null  object 
 5   State             286392 non-null  object 
 6   price             286392 non-null  float64
 7   value             286392 non-null  float64
 8   Discount_Percent  286392 non-null  float64
 9   discount_amount   286392 non-null  float64
 10  total             286392 non-null  float64
 11  qty_ordered       286392 non-null  float64
dtypes: float64(7), object(5)
memory usage: 28.4+ MB


Categorical Features:
- month
- category
- gender
- region
- state

Label Encoding

In [143]:
encoder = LabelEncoder()

In [144]:
cat_features = ['month', 'category', 'Gender', 'Region', 'State']

for cat_feature in cat_features:
    dataset[cat_feature] = encoder.fit_transform(dataset[cat_feature])

c:\Users\Asus\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [145]:
dataset

,month,category,Gender,age,Region,State,price,value,Discount_Percent,discount_amount,total,qty_ordered
0,10,8,0,43.0,2,36,89.9,1798.0,0.0,0.0,1798.0,22.0
1,10,8,0,43.0,2,36,19.0,190.0,0.0,0.0,190.0,12.0
2,10,8,0,43.0,2,36,149.9,1199.2,0.0,0.0,1199.2,10.0
3,10,8,0,43.0,2,36,79.9,639.2,0.0,0.0,639.2,10.0
4,9,8,0,43.0,2,36,99.9,99.9,0.0,0.0,99.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
286387,11,0,1,51.0,1,34,4419.9,4419.9,0.0,0.0,4419.9,3.0
286388,11,8,1,52.0,2,10,39.9,39.9,0.0,0.0,39.9,3.0
286389,11,14,0,38.0,2,27,40.0,40.0,0.0,0.0,40.0,3.0
286390,11,14,0,38.0,2,27,49.9,49.9,0.0,0.0,49.9,3.0


Split train and test dataset

In [146]:
X = dataset.drop(columns='qty_ordered', axis=1)
Y = dataset['qty_ordered']

In [147]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

Model Training

In [148]:
regressor = XGBRegressor()

In [149]:
regressor.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

Evaluation

In [150]:
# prediction on training data
training_data_prediction = regressor.predict(X_train)
# R squared Value
r2_train = metrics.r2_score(Y_train, training_data_prediction)
print('R Squared value = ', r2_train)

R Squared value =  0.9994047945928058


In [151]:
# prediction on test data
test_data_prediction = regressor.predict(X_test)
# R squared Value
r2_test = metrics.r2_score(Y_test, test_data_prediction)
print('R Squared value = ', r2_test)

R Squared value =  0.9861241953554252
